# Biostat M280 Homework 3

**Due Friday, May 25 @ 11:59PM**

## Q1 - Big $n$ regression

Those who took my _203B: Introduction to Data Science_ last quarter had a (painful) experience of wrangling an Apache Spark cluster to do linear regression on a dataset with more than 100 million observations. Now we learnt various methods for solving linear regression and should realize that, with right choice of algorithm, it is a problem that can be handled by any moderate computer.

### Q1(1)

Download the flight data from <http://stat-computing.org/dataexpo/2009/the-data.html>. For this exercise, we only need data from years 2003-2008. If you are using Mac or Linux, you can run the following Bash script, which downloads and unzips files for all years.
```bash
# Download flight data by year
for i in {1987..2008}
  do
    echo "$(date) $i Download"
    fnam=$i.csv.bz2
    wget -O ./$fnam http://stat-computing.org/dataexpo/2009/$fnam
    echo "$(date) $i unzip"
    bzip2 -d ./$fnam
  done

# Download airline carrier data
wget -O ./airlines.csv http://www.transtats.bts.gov/Download_Lookup.asp?Lookup=L_UNIQUE_CARRIERS

# Download airports data
wget -O ./airports.csv https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat
```
Find out how many data points in each year.

In [42]:
@show c2003 = countlines("2003.csv")
@show c2004 = countlines("2004.csv")
@show c2005 = countlines("2005.csv")
@show c2006 = countlines("2006.csv")
@show c2007 = countlines("2007.csv")
@show c2008 = countlines("2008.csv")
@show total_lines = c2003 + c2004 + c2005 + c2006 + c2007 + c2008

c2003 = countlines("2003.csv") = 6488541
c2004 = countlines("2004.csv") = 7129271
c2005 = countlines("2005.csv") = 7140597
c2006 = countlines("2006.csv") = 7141923
c2007 = countlines("2007.csv") = 7453216
c2008 = countlines("2008.csv") = 7009729
total_lines = c2003 + c2004 + c2005 + c2006 + c2007 + c2008 = 42363277


42363277

### Q1(2) 

We are interested in how the time gain of a flight, defined as `DepDelay - ArrDelay`, depends on the distance traveled (`Distance`), departure delay (`DepDelay`), and carrier (`UniqueCarrier`). 

We want to fit a linear regression `Gain ~ 1 + Distance + DepDelay + UniqueCarrier` using data from 2003-2008. Note `UniqueCarrier` is a factor with 23 levels: "9E", "AA", "AQ", "AS", "B6", "CO", "DH", "DL", "EV", "F9", "FL", "HA", "HP", "MQ", "NW", "OH", "OO", "TZ", "UA", "US", "WN", "XE", and "YV". We use the dummy coding with "9E" as base level.

Will the design matrix (in double precision) fit into the memory of you computer?

**(2)** Data from 2003-2008 contain 42363277 observations. The design matrix has dimension $42363277 \times 26$, so in total $42363277 \times 26$ of double-precision numbers. Every double-precision number is 8 bytes. We need a memory of $42363277 \times 26 \times 8 / 10^9 \sim $8.81GB to fit the whole matrix. My poor MacBook with a 8GB memory can not handle it.

### Q1(3)

Review the [Summary of Linear Regression](http://hua-zhou.github.io/teaching/biostatm280-2018spring/slides/12-linreg/linreg.html) and devise a strategy to solve the linear regression.

Report the estimated regression coefficients $\widehat \beta$, estimated variance $\widehat \sigma^2 = \sum_i (y_i - \widehat y_i)^2 / (n - 1)$, and coefficient standard errors.

Hint: It took my laptop less than 3 minutes to import data and fit linear regression.

In [46]:
using SweepOperator

srand(280)

X = randn(5, 3) # predictor matrix
y = randn(5)    # response vector

# form the augmented Gram matrix
G = [X y]' * [X y]

4×4 Array{Float64,2}:
  9.589    -4.98208  -5.70052  -1.83933
 -4.98208   4.94476   3.81663   2.82462
 -5.70052   3.81663   5.45442   2.46008
 -1.83933   2.82462   2.46008   4.98343

In [8]:
# mapping from variable names to X columns
# carrier "9E" is used as base level
const var2col = Dict(
        "Intercept" => 1,
        "Distance" => 2,
        "DepDelay" => 3,
        "AA" => 4,
        "AQ" => 5,
        "AS" => 6,
        "B6" => 7,
        "CO" => 8,
        "DH" => 9,
        "DL" => 10,
        "EV" => 11,
        "F9" => 12,
        "FL" => 13,
        "HA" => 14,
        "HP" => 15,
        "MQ" => 16,
        "NW" => 17,
        "OH" => 18,
        "OO" => 19,
        "TZ" => 20,
        "UA" => 21,
        "US" => 22,
        "WN" => 23,
        "XE" => 24,
        "YV" => 25,
        "Gain" => 26)
# mapping from column to variable names
const col2var = map(reverse, var2col)

# a custom function to generate [X y] from data table
function generate_xy(tbl::NextTable)
    # X matrix
    XY = zeros(length(tbl), 26)
    # intercept term
    @views fill!(XY[:, 1], 1)
    # Distance term
    @views copy!(XY[:, 2], columns(tbl, :Distance))
    # DepDelay term
    @views copy!(XY[:, 3], columns(tbl, :DepDelay))
    # Dummy coding for airline
    @inbounds for i in 1:length(tbl)
        tbl[i][:UniqueCarrier] == "9E" && continue # base level
        XY[i, var2col[tbl[i][:UniqueCarrier]]] = 1
    end
    # last column is response: gain = depdelay - arrdelay
    XY[:, 26] = select(tbl, 
        (:DepDelay, :ArrDelay) => p -> Float64(p.DepDelay - p.ArrDelay))
    # return
    XY
end

generate_xy (generic function with 1 method)

In [1]:
using JuliaDB
# only need columns: DepDelay, ArrDelay, UniqueCarrier, Distance
@time yrtable2003 = loadtable(
    "2003.csv", 
    datacols = ["DepDelay", "ArrDelay", "UniqueCarrier", "Distance"])
@time yrtable2004 = loadtable(
    "2004.csv", 
    datacols = ["DepDelay", "ArrDelay", "UniqueCarrier", "Distance"])

 48.905613 seconds (153.54 M allocations: 8.160 GiB, 25.52% gc time)
 34.437851 seconds (31.53 M allocations: 4.276 GiB, 6.43% gc time)


Table with 7129270 rows, 4 columns:
DepDelay  ArrDelay  UniqueCarrier  Distance
───────────────────────────────────────────
-7        -14       "UA"           599
-9        -4        "UA"           599
3         5         "UA"           599
-3        -16       "UA"           599
5         3         "UA"           599
-2        -10       "UA"           599
20        29        "UA"           599
-3        -11       "UA"           599
-7        -12       "UA"           599
-4        -14       "UA"           599
1         8         "UA"           599
-2        -14       "UA"           599
⋮
0         -4        "DL"           821
-2        -9        "DL"           590
-2        -10       "DL"           532
-1        6         "DL"           391
2         -15       "DL"           599
-6        -15       "DL"           215
6         -1        "DL"           425
6         -1        "DL"           1541
6         3         "DL"           547
-2        -3        "DL"           545
-2        -13  

In [14]:
using SweepOperator
yrtable2003 = dropna(yrtable2003)
yrtable2004 = dropna(yrtable2004)
g3 = generate_xy(yrtable2003)
g4 = generate_xy(yrtable2004)
g = g3'*g3 
g

26×26 Array{Float64,2}:
      6.37569e6  4.5497e9         3.32999e7   …  0.0        1.03685e7 
      4.5497e9   5.28186e12       2.37964e10     0.0        1.01318e10
      3.32999e7  2.37964e10       4.51178e9      0.0        4.30844e7 
 738567.0        7.7143e8         3.42572e6      0.0        1.05689e6 
      0.0        0.0              0.0            0.0        0.0       
 157602.0        1.31742e8   895388.0         …  0.0   394580.0       
  66732.0        8.49327e7   377211.0            0.0   311411.0       
 299843.0        3.24764e8        1.1675e6       0.0    33212.0       
 280010.0        1.00559e8        2.63428e6      0.0   804746.0       
 652602.0        5.2721e8         2.95877e6      0.0   442798.0       
 266760.0        1.08515e8        2.37215e6   …  0.0   595560.0       
      0.0        0.0              0.0            0.0        0.0       
 142817.0        8.51543e7   983078.0            0.0  -211739.0       
   7792.0        4.77305e6     8050.0            0.0 

In [13]:
sweep!(g, 1:25)

26×26 Array{Float64,2}:
    NaN          NaN           …  NaN              NaN    NaN    NaN
      4.5497e9   NaN              NaN              NaN    NaN    NaN
      3.32999e7    2.37964e10     NaN              NaN    NaN    NaN
 738567.0          7.7143e8       NaN              NaN    NaN    NaN
      0.0          0.0            NaN              NaN    NaN    NaN
 157602.0          1.31742e8   …  NaN              NaN    NaN    NaN
  66732.0          8.49327e7      NaN              NaN    NaN    NaN
 299843.0          3.24764e8      NaN              NaN    NaN    NaN
 280010.0          1.00559e8      NaN              NaN    NaN    NaN
 652602.0          5.2721e8       NaN              NaN    NaN    NaN
 266760.0          1.08515e8   …  NaN              NaN    NaN    NaN
      0.0          0.0            NaN              NaN    NaN    NaN
 142817.0          8.51543e7      NaN              NaN    NaN    NaN
   7792.0          4.77305e6      NaN              NaN    NaN    NaN
 187266.0 

In [18]:
# maybe a function
using JuliaDB
# pre-allocate a matrix gram_augmented  
n = 0
p = 26

gram_augmented = zeros(p, p)
gram_augmented_sum = zeros(p, p)
@time for year = 2003:2008
    filename = join([string(year),".csv"]) 
    yrtable = loadtable(
        filename, 
        datacols = ["DepDelay", "ArrDelay", "UniqueCarrier", "Distance"])
    yrtable = dropna(yrtable)
    xy_matrix = generate_xy(yrtable)
    n += size(xy_matrix, 1)
    gram_augmented = At_mul_B(xy_matrix, xy_matrix)
    gram_augmented_sum += gram_augmented
end

247.965442 seconds (483.02 M allocations: 44.730 GiB, 18.21% gc time)


In [21]:
using SweepOperator
sweep_result = sweep!(gram_augmented_sum, 1:(p - 1))

26×26 Array{Float64,2}:
     -2.00254e-6   4.21792e-11  …   1.98347e-6   -1.14033   
      3.00028e10  -9.38376e-14     -5.01596e-12  -0.00164935
      3.69821e8    2.76146e11       1.16836e-10   0.0118811 
      3.91626e6    4.19149e9       -1.97927e-6    1.8723    
  88336.0          3.75197e7       -1.98134e-6    0.5789    
 937547.0          8.30881e8    …  -1.98009e-6    0.938452  
 799117.0          9.55599e8       -1.97878e-6    1.42247   
      1.8094e6     2.03781e9       -1.97887e-6    2.57627   
 669687.0          2.51484e8       -1.98271e-6   -1.16808   
      3.37715e6    2.9514e9        -1.97993e-6    2.19625   
      1.64509e6    7.42218e8    …  -1.98278e-6   -1.03932   
 334842.0          2.97744e8       -1.97972e-6    2.15207   
      1.24932e6    8.33171e8       -1.98132e-6    1.35247   
 272859.0          1.61374e8       -1.98044e-6    1.87248   
 574927.0          5.84803e8       -1.97917e-6    0.350758  
      2.90986e6    1.13812e9    …  -1.98262e-6    1.46395   


**(3) continued ** Recall that sweeping on the Gram matrix yields 

$$
\begin{eqnarray*}
\begin{pmatrix}
- (\mathbf{X}^T \mathbf{X})^{-1} & (\mathbf{X}^T \mathbf{X})^{-1} \mathbf{X}^T \mathbf{y} \\
\mathbf{y}^T \mathbf{X} (\mathbf{X}^T \mathbf{X})^{-1} & \mathbf{y}^T \mathbf{y} - \mathbf{y}^T \mathbf{X} (\mathbf{X}^T \mathbf{X})^{-1} \mathbf{X}^T \mathbf{y}
\end{pmatrix} = 
\begin{pmatrix}
- \sigma^{-2} \text{Cov}(\widehat \beta) & \widehat \beta \\
\widehat \beta^T & \|\mathbf{y} - \hat y\|_2^2
\end{pmatrix}.
\end{eqnarray*}
$$ 

The estimated regression coefficients $\widehat \beta$, estimated variance $\widehat \sigma^2 = \sum_i (y_i - \widehat y_i)^2 / (n - 1)$, and coefficient standard errors can be easily gained from the sweeped matrix.

$\widehat \beta =$

In [24]:
sweepresult[1:p - 1, p]

25-element Array{Float64,1}:
 -1.14033   
 -0.00164935
  0.0118811 
  1.8723    
  0.5789    
  0.938452  
  1.42247   
  2.57627   
 -1.16808   
  2.19625   
 -1.03932   
  2.15207   
  1.35247   
  1.87248   
  0.350758  
  1.46395   
  3.62506   
  0.00722279
  0.40365   
  3.5774    
  1.14816   
  0.883753  
 -2.74855   
  2.56721   
  0.202211  

$\widehat \sigma^2 = \sum_i (y_i - \widehat y_i)^2 / (n - 1) = $

In [25]:
sigma2 = sweepresult[p, p] / (n - 1)

204.40310742569457

Coefficient standard errors

In [33]:
sqrt.(diag(sweepresult[1:p-1, 1:p-1] * (-sigma2)))

25-element Array{Float64,1}:
 0.0202318 
 4.37958e-6
 6.88878e-5
 0.0215571 
 0.0521477 
 0.0250361 
 0.0259154 
 0.0229534 
 0.0266552 
 0.021659  
 0.023012  
 0.0319265 
 0.023868  
 0.0339852 
 0.0276918 
 0.0218055 
 0.0220088 
 0.023446  
 0.0217463 
 0.0375073 
 0.0219444 
 0.0220022 
 0.0209201 
 0.0222383 
 0.0255715 

### Q1(4)

Go to your resume/cv and claim you have experience performing analytics on data with hundred millions of observations.

## Q2 - Google PageRank

We are going to try different numerical methods learnt in class on the [Google PageRank problem](https://en.wikipedia.org/wiki/PageRank).

### Q2(1)

Let $\mathbf{A} \in \{0,1\}^{n \times n}$ be the connectivity matrix of $n$ web pages with entries
$$
\begin{eqnarray*}
	a_{ij}= \begin{cases}
	1 & \text{if page $i$ links to page $j$} \\
	0 & \text{otherwise}
	\end{cases}.
\end{eqnarray*}
$$
$r_i = \sum_j a_{ij}$ is the out-degree of page $i$. That is $r_i$ is the number of links on page $i$. Imagine a random surfer exploring the space of $n$ pages according to the following rules.  

- From a page $i$ with $r_i>0$
    * with probability $p$, (s)he randomly chooses a link on page $i$ (uniformly) and follows that link to the next page  
    * with probability $1-p$, (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page  
- From a page $i$ with $r_i=0$ (a dangling page), (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page  
    
The process defines a Markov chain on the space of $n$ pages. Write the transition matrix $\mathbf{P}$ of the Markov chain as a sparse matrix plus rank 1 matrix.

### Q2(2)

According to standard Markov chain theory, the (random) position of the surfer converges to the stationary distribution $\mathbf{x} = (x_1,\ldots,x_n)^T$ of the Markov chain. $x_i$ has the natural interpretation of the proportion of times the surfer visits page $i$ in the long run. Therefore $\mathbf{x}$ serves as page ranks: a higher $x_i$ means page $i$ is more visited. It is well-known that $\mathbf{x}$ is the left eigenvector corresponding to the top eigenvalue 1 of the transition matrix $\mathbf{P}$. That is $\mathbf{P}^T \mathbf{x} = \mathbf{x}$. Therefore $\mathbf{x}$ can be solved as an eigen-problem. Show that it can also be cast as solving a linear system. Since the row sums of $\mathbf{P}$ are 1, $\mathbf{P}$ is rank deficient. We can replace the first equation by the $\sum_{i=1}^n x_i = 1$.

$(\mathbf{P}^T -\mathbf{I} )\mathbf{x} = \mathbf{0}$

### Q2(3)

Download the [`ucla.zip`](http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw3/ucla.zip) package from course webpage. Unzip the package, which contains two files `U.txt` and `A.txt`. `U.txt` lists the 500 URL names. `A.txt` is the $500 \times 500$ connectivity matrix. Read data into Julia. Compute summary statistics:
* number of pages
* number of edges
* number of dangling nodes (pages with no out links)
* which page has max in-degree?
* which page has max out-degree?
* visualize the sparsity pattern of $\mathbf{A}$

### Q2(4)

Set the _teleportation_ parameter at $p = 0.85$. Try the following methods to solve for $\mathbf{x}$ using the `ucla.zip` data.

0. A dense linear system solver such as LU decomposition.  
0. A simple iterative linear system solver such as Jacobi or Gauss-Seidel.   
0. A dense eigen-solver.  
0. A simple iterative eigen-solver such as the power method.  

For iterative methods, you can use the [`IterativeSolvers.jl`](https://github.com/JuliaMath/IterativeSolvers.jl) package. Make sure to utilize the special structure of $\mathbf{P}$ (sparse + rank 1) to speed up the matrix-vector multiplication.

### Q2(5)

List the top 20 ranked URLs you found.

### Q2(6)

As of Monday May 11 2018, there are at least 1.83 billion indexed webpages on internet according to <http://www.worldwidewebsize.com/>. Explain whether each of these methods works for the PageRank problem at this scale.